In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.2. Biomarker Cluster Analysis/Output')
cur_date = "041423"

library(readxl)
library(tidyverse)
library(reshape2)
library(factoextra)

# reading in files
mRNA_df = data.frame(read_excel("Input/mRNA_Data_041423.xlsx", sheet = 2))
proteomics_5_df = data.frame(read_excel("Input/Proteomics_Data_5ug_040423.xlsx", sheet = 2))
proteomics_25_df = data.frame(read_excel("Input/Proteomics_Data_25ug_040423.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa



In [2]:
head(mRNA_df)
head(proteomics_25_df)

,Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,ALDH1A3,C,F,1,24,-6.6824749,4.480105
2,2,M_2,ALDH1A3,C,F,1,24,0.2560009,4.870661
3,3,M_3,ALDH1A3,C,F,1,24,0.4318636,4.879307
4,4,F_4,ALDH1A3,C,F,1,24,0.4193253,4.878692
5,6,M_6,ALDH1A3,C,F,1,24,-0.1550695,4.850246
6,1,F_1,ALDH1A3,C,F,1,72,-0.2119125,4.847400


,Subject_No,Subject_ID,Protein_Accession,Gene_Name,Description,Condensate,Burn_Condition,Concentration,Intensity,Intensity_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,1488600000,30.47131
2,2,M_2,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3806600000,31.82586
3,3,M_3,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3900000000,31.86083
4,4,F_4,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5373200000,32.32313
5,5,F_5,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,2595200000,31.27320
6,6,M_6,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5780700000,32.42860


This analysis will use PCA & k means to explore to determine if each biomarker clusters into groups.

In [33]:
# dim(mRNA_df)
# mRNA_df[sort(duplicated(mRNA_df[,1:7])),]

test = mRNA_df %>%#[duplicated(mRNA_df[,1:7]),] %>%
    group_by(mRNA, Subject_ID, Condensate, Burn_Condition, Concentration, Time_Point) %>%
    #add_tally()
    mutate(count = n()) %>% 
    arrange(-count) #%>%
    #mutate(count2 = ifelse(count == 2, 1:count, NA))
head(test)

Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2,count
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>
6,M_6,HMOX1,PBS,PBS,NA,24,0.70848207,4.892803,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.36651641,4.876100,6
6,M_6,HMOX1,PBS,PBS,NA,24,-0.09090465,4.853452,6
6,M_6,HMOX1,PBS,PBS,NA,24,-0.37654696,4.839126,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.72283828,4.893500,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.37135970,4.876338,6


In [63]:
# WORDS
replicate_index_count_df = data.frame()
for(i in 1:length(test$Subject_ID)){
    # taking care of edge case
    if(i > 1){
        # if the value of the count is less than the one before it do this
        if(test$count[i] < test$count[i - 1]){
            # saving index and replicate number
            index = i - 1
            count_number = test$count[i - 1]
            replicate_index_count_df = rbind(replicate_index_count_df, c(index, count_number))

        }
    }
}

# adding column names
colnames(replicate_index_count_df) = c("Index", "Replicate_Count")

replicate_index_count_df

Index,Replicate_Count
<dbl>,<dbl>
6,6
86,5
414,4
510,3
1850,2


In [90]:
# these numbers exactly what we need...
# the previous index needs to be subtracted from the next and divided by the replicate count

vector = c()
for (i in 1:length(replicate_index_count_df$Index)){
    if (i > 1){
        new_index = (replicate_index_count_df$Index[i] - replicate_index_count_df$Index[i - 1])/ replicate_index_count_df$Replicate_Count[i]
        #print(new_index)
        Replicate_Iteration = rep(1:replicate_index_count_df$Replicate_Count[i], new_index)
        vector = c(vector, length(Replicate_Iteration))
    }
}

sum(vector)

[1] 1844

In [83]:
# # creating a column based on the above df
# Replicate_Number = c()
# for (i in 1:length(replicate_index_count_df$Index)){
#     Replicate_Iteration = rep(1:replicate_index_count_df$Replicate_Count[i], 
#                               times = replicate_index_count_df$Index[i])

#     Replicate_Number = c(Replicate_Number, Replicate_Iteration)
# }

# length(Replicate_Number)
length(rep(1:replicate_index_count_df$Replicate_Count[2], 
           times = (replicate_index_count_df$Index[2] - replicate_index_count_df$Index[1])/ replicate_index_count_df$Replicate_Count[2]))

[1] 80

In [84]:
test %>%
    filter(count == 6)

Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2,count
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>
6,M_6,HMOX1,PBS,PBS,NA,24,0.70848207,4.892803,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.36651641,4.876100,6
6,M_6,HMOX1,PBS,PBS,NA,24,-0.09090465,4.853452,6
6,M_6,HMOX1,PBS,PBS,NA,24,-0.37654696,4.839126,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.72283828,4.893500,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.37135970,4.876338,6


In [21]:
# pivoting longer but first creating sample ids
wider_mRNA_df = mRNA_df %>%
    # creating a column that denotes the biomarker
    mutate(Biomarker = "mRNA") %>%
    # creating a column for the sample number to deal with replicates
    #mutate(Sample_No = 1:dim(mRNA_df)[1]) %>%
    # creating sample ids
    unite("Sample_ID", c(colnames(mRNA_df)[c(4:7)], "Biomarker")) %>%
    select(-c("Subject_No", "ddCT")) %>%
    pivot_wider(names_from = "Subject_ID", values_from = "ddCT_pslog2") %>%
    filter(mRNA == unique(mRNA_df$mRNA)[1])

wider_mRNA_df

Warning message:
“Values from `ddCT_pslog2` are not uniquely identified; output will contain list-cols.
* Use `values_fn = list` to suppress this warning.
* Use `values_fn = {summary_fun}` to summarise duplicates.
* Use the following dplyr code to identify duplicates.
  {data} %>%
    dplyr::group_by(mRNA, Sample_ID, Subject_ID) %>%
    dplyr::summarise(n = dplyr::n(), .groups = "drop") %>%
    dplyr::filter(n > 1L)”


mRNA,Sample_ID,F_1,M_2,M_3,F_4,M_6,F_5
<chr>,<chr>,<list>,<list>,<list>,<list>,<list>,<list>
ALDH1A3,C_F_1_24_mRNA,4.480105,4.870661,4.879307,4.878692,4.850246,NULL
ALDH1A3,C_F_1_72_mRNA,4.8474,4.853069,4.850874,4.881133,4.844027,4.877173
ALDH1A3,C_S_1_24_mRNA,NULL,4.842062,4.869919,4.863741,4.91511,4.902314
ALDH1A3,C_S_1_72_mRNA,4.875436,4.885058,4.868836,4.858839,4.885779,4.879357
ALDH1A3,C_F_25_4_mRNA,"4.850829, 4.843997","4.858401, 4.862782","4.898830, 4.887884","4.861283, 4.873939","4.861486, 4.857343",NULL
ALDH1A3,C_F_25_24_mRNA,4.891977,4.881552,4.863584,4.861217,4.875984,4.877587
ALDH1A3,C_F_25_72_mRNA,4.904412,4.89836,4.849387,4.874381,4.865843,4.868691
ALDH1A3,C_S_25_4_mRNA,"4.910852, 4.895969","4.864358, 4.861943","4.891048, 4.885630","4.867805, 4.871020","4.892004, 4.882764","4.860395, 4.862442"
ALDH1A3,C_S_25_24_mRNA,4.865115,4.842453,4.851229,4.913267,4.851951,4.898058


In [32]:
head(proteomics_25_df)
head(proteomics_5_df)

,Subject_No,Subject_ID,Protein_Accession,Gene_Name,Description,Condensate,Burn_Condition,Concentration,Intensity,Intensity_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,1488600000,30.47131
2,2,M_2,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3806600000,31.82586
3,3,M_3,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3900000000,31.86083
4,4,F_4,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5373200000,32.32313
5,5,F_5,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,2595200000,31.27320
6,6,M_6,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5780700000,32.42860


,Subject_No,Subject_ID,Protein_Accession,Gene_Name,Description,Condensate,Burn_Condition,Concentration,Intensity,Intensity_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,5,1.16e+09,30.11148
2,2,M_2,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,5,1.29e+09,30.26472
3,3,M_3,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,5,2.87e+09,31.41840
4,5,F_5,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,5,2.91e+09,31.43837
5,6,M_6,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,5,5.96e+09,32.47267
6,4,F_4,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,5,4.57e+09,32.08955


In [33]:
wider_proteomics_25_df = proteomics_25_df %>%
    # creating a column that denotes the biomarker
    mutate(Biomarker = "Protein") %>%
    # creating sample ids
    unite("Sample_ID", c(colnames(proteomics_25_df)[c(3,6:8)], "Biomarker")) %>%
    select(-colnames(proteomics_25_df)[c(1,4,5,9)]) %>%
    pivot_wider(names_from = "Subject_ID", values_from = "Intensity_pslog2")

wider_proteomics_5_df = proteomics_5_df %>%
    # creating a column that denotes the biomarker
    mutate(Biomarker = "Protein") %>%
    # creating sample ids
    unite("Sample_ID", c(colnames(proteomics_5_df)[c(3,6:8)], "Biomarker")) %>%
    select(-colnames(proteomics_5_df)[c(1,4,5,9)]) %>%
    pivot_wider(names_from = "Subject_ID", values_from = "Intensity_pslog2")

# creating 1 df
wider_proteomics_df = rbind(wider_proteomics_25_df, wider_proteomics_5_df)
head(wider_proteomics_df)

Sample_ID,F_1,M_2,M_3,F_4,F_5,M_6
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CO3_HUMAN_C_F_25_Protein,30.47131,31.82586,31.86083,32.32313,31.27320,32.42860
CO3_HUMAN_C_S_25_Protein,31.95699,31.17766,32.27534,33.11253,31.32839,31.60576
CO3_HUMAN_PBS_PBS_25_Protein,28.35565,30.75581,31.88987,32.22239,31.53371,33.69975
CO3_HUMAN_P_F_25_Protein,31.09012,31.49596,32.07058,33.27462,31.81062,32.34631
CO3_HUMAN_P_S_25_Protein,31.06664,31.42337,32.18299,33.80097,31.79819,33.90464
K1C19_HUMAN_C_F_25_Protein,32.48738,34.19938,31.95404,32.58571,32.79860,31.79231
